<a href="https://colab.research.google.com/github/sboomi/exploradome_tangram/blob/tf2---team-3/Jason_train_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows how to classify cats or dogs from images. It builds an image classifier using a `tf.keras.Sequential` model and load data using `tf.keras.preprocessing.image.ImageDataGenerator`. You will get some practical experience and develop intuition for the following concepts:

* Building _data input pipelines_ using the `tf.keras.preprocessing.image.ImageDataGenerator` class to efficiently work with data on disk to use with the model.
* _Overfitting_ —How to identify and prevent it.
* _Data augmentation_ and _dropout_ —Key techniques to fight overfitting in computer vision tasks to incorporate into the data pipeline and image classifier model.

This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import packages

Let's start by importing the required packages. The `os` package is used to read files and directory structure, NumPy is used to convert python list to numpy array and to perform required matrix operations and `matplotlib.pyplot` to plot the graph and display images in the training and validation data.

Import Tensorflow and the Keras classes needed to construct our model.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

import os
import numpy as np
import matplotlib.pyplot as plt

## Load data

Begin by downloading the dataset. This tutorial uses a filtered version of <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a> dataset from Kaggle. Download the archive version of the dataset and store it in the "/tmp/" directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

After extracting its contents, assign variables with the proper file path for the training and validation set.

In [ ]:
import os

PATH = '/content/drive/My Drive/data/'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

### Understand the data

Let's look at how many cats and dogs images are in the training and validation directory:

In [ ]:
num_tr = len(os.listdir(train_dir))
num_val = len(os.listdir(validation_dir))

In [ ]:
print(os.listdir(train_dir))

In [ ]:
num_tr

For convenience, set up variables to use while pre-processing the dataset and training the network.

In [ ]:
batch_size = 3
epochs = 15
IMG_HEIGHT = 224
IMG_WIDTH = 224

## Data preparation

Format the images into appropriately pre-processed floating point tensors before feeding to the network:

1. Read images from the disk.
2. Decode contents of these images and convert it into proper grid format as per their RGB content.
3. Convert them into floating point tensors.
4. Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values.

Fortunately, all these tasks can be done with the `ImageDataGenerator` class provided by `tf.keras`. It can read images from disk and preprocess them into proper tensors. It will also set up generators that convert these images into batches of tensors—helpful when training the network.

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=90,
                    horizontal_flip=True,
                    vertical_flip=True
                    )

In [ ]:
train_image_generator = image_gen_train # Generator for our training data
validation_image_generator = image_gen_train # Generator for our validation data

After defining the generators for training and validation images, the `flow_from_directory` method load images from the disk, applies rescaling, and resizes the images into the required dimensions.

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size, #directory containing various tangram
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH), # all images will be resized to 150, 150 when it is loaded
                                                           class_mode='categorical') #categorial because of multi-class classification. 
                                                                                      #if only two classes, then "binay"

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

### Visualize training images

Visualize the training images by extracting a batch of images from the training generator—which are 128 images in this example—then plot five of them with `matplotlib`.

In [ ]:
sample_training_images, _ = next(train_data_gen)

The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is training features and y_train, its labels. Discard the labels to only visualize the training images.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

## Create the model

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False
base_model.summary()


In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
prediction_layer = tf.keras.layers.Dense(12, activation='softmax')

In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

## Compile the model

We choose the rmsprop optimizer and categorical_crossentropy loss function. To view training and validation accuracy for each training epoch, pass the metrics argument.

In [ ]:
lr = 0.0001
model.compile(optimizer=tf.optimizers.RMSprop(),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
initial_epochs = 10
validation_steps=20

loss0,accuracy0 = model.evaluate(val_data_gen, steps = validation_steps)

## Train the model

In [ ]:
history = model.fit(train_data_gen,
                    epochs=100,
                    callbacks=[callback],
                    validation_data=val_data_gen)

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 512 units on top of it that is activated by a `relu` activation function.

In [ ]:
filepath = r"/content/drive/My Drive/saved_model"
tf.keras.models.save_model(
    model,
    filepath,
    overwrite=True,
    include_optimizer=True,
    save_format="tf",
    signatures=None
)

model.save("tangram_jason_mobile_net.h5")

# another way to save a model 
# https://www.tensorflow.org/tutorials/keras/save_and_load

### Visualize training results

Now visualize the results after training the network.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(100)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc[-1], val_acc[-1], loss[-1], val_loss[-1]

## Definition for prediction

In [ ]:
import time
import cv2

In [ ]:
def result_image(path):
  start_time = time.time()
  img = cv2.imread(image_test)
  height, width, channels = img.shape
  width_cutoff = width // 2
  s1 = img[:, :width_cutoff]
  s2 = img[:, width_cutoff:]
  s1_up = tf.image.resize(s1/255, (224,224), preserve_aspect_ratio=False)
  s1_final = tf.expand_dims(s1_up, axis=0)
  s2_up = tf.image.resize(s2/255, (224,224), preserve_aspect_ratio=False)
  s2_final = tf.expand_dims(s2_up, axis=0)
  result_1 = model.predict(s1_final)
  result_dict_1 = {}
  result_2 = model.predict(s2_final)
  result_dict_2 = {}
  for i in range(result_1.shape[1]):
    result_dict_1[os.listdir(train_dir)[i]] = (str(result_1[0][i])+" %")
    result_dict_2[os.listdir(train_dir)[i]] = (str(result_2[0][i])+" %")
  end_time = time.time()
  total_fps = 1/(end_time-start_time)
  print("Total time:",end_time-start_time)
  print("FPS:",total_fps)
  return result_dict_1, result_dict_2

In [ ]:
result_image(image_test)

In [ ]:
image_test